Ce script est fait pour tester la calibration politique du model Openai 4.1-mini
Il suffit de mettre une url et d'exécuter la requête

In [ ]:
# ========================================
# TEST: TRANSCRIPTION + ANALYSE D'UN SEUL MP3
# ========================================
import requests
import json
import time
import re

# ========================================
# CONFIGURATION
# ========================================
gladia_key = 'XXXXX'
openai_key = 'XXXXXX'

# ⬇️ METTEZ VOTRE URL MP3 ICI
TEST_MP3_URL = "https://drive.google.com/uc?export=download&id=1mnXKjmyH_e_9QysvopbhfBY_PmYqBIHs"
#melenchon: TEST_MP3_URL = "https://drive.google.com/uc?export=download&id=1dl004eMWiYdC2jlVEtzfWAH38trxhKeu"
#ciotti TEST_MP3_URL = "https://drive.google.com/uc?export=download&id=1jiRkNgEPBm3EU1R3Casmv3OkCBjobcuT"
#macron TEST_MP3_URL = "https://drive.google.com/uc?export=download&id=1mnXKjmyH_e_9QysvopbhfBY_PmYqBIHs"



# ========================================
# ÉTAPE 1: TRANSCRIPTION AVEC GLADIA
# ========================================
print("="*60)
print("ÉTAPE 1: TRANSCRIPTION")
print("="*60)

# Lancer la transcription
print(f"\n📤 Envoi à Gladia: {TEST_MP3_URL[:60]}...")

init_response = requests.post(
    'https://api.gladia.io/v2/pre-recorded',
    headers={
        'x-gladia-key': gladia_key,
        'Content-Type': 'application/json'
    },
    json={
        'audio_url': TEST_MP3_URL,
        'language': 'fr',
        'diarization': True,
        'diarization_config': {
            'min_speakers': 1,
            'max_speakers': 10,
        }
    },
    timeout=60
)
init_response.raise_for_status()
result_url = init_response.json().get('result_url')
print(f"✅ Transcription lancée, polling: {result_url[:50]}...")

# Polling
status = 'processing'
poll_count = 0
while status in ['queued', 'processing'] and poll_count < 60:
    time.sleep(10)
    poll_count += 1
    poll_response = requests.get(result_url, headers={'x-gladia-key': gladia_key}, timeout=30)
    result = poll_response.json()
    status = result.get('status')
    print(f"   ⏳ Polling #{poll_count}: {status}")

if status != 'done':
    print(f"❌ Échec transcription: {status}")
    exit()

# Extraire la transcription
transcription_data = result.get('result', {}).get('transcription', {})
utterances = transcription_data.get('utterances', [])

# Formater avec speakers
lines = []
current_speaker = None
for utt in utterances:
    speaker = utt.get('speaker', 'Unknown')
    text = utt.get('text', '').strip()
    if speaker != current_speaker:
        lines.append(f"\n[Speaker {speaker}]: {text}")
        current_speaker = speaker
    else:
        lines[-1] += f" {text}"

transcript = ''.join(lines).strip()
num_speakers = len(set(u.get('speaker') for u in utterances))

print(f"\n✅ TRANSCRIPTION TERMINÉE")
print(f"   Longueur: {len(transcript)} caractères")
print(f"   Speakers: {num_speakers}")
print(f"\n{'='*60}")
print("EXTRAIT DE LA TRANSCRIPTION (500 premiers caractères):")
print("="*60)
print(transcript[:500] + "..." if len(transcript) > 500 else transcript)

# ========================================
# ÉTAPE 2: ANALYSE POLITIQUE AVEC GPT
# ========================================
print(f"\n{'='*60}")
print("ÉTAPE 2: ANALYSE POLITIQUE")
print("="*60)

SYSTEM_PROMPT = """Tu es un analyste média et politique français expert.

## OBJECTIF
Analyser le BIAIS ÉDITORIAL et l'ORIENTATION POLITIQUE de chaque intervenant (speaker) dans cette transcription.

## REGLE CRITIQUE
Ne tiens pas compte du nom des intervenants. N'analyse que le contenu.

## CATÉGORIES POLITIQUES FRANÇAISES
- Extrême gauche (LFI, NPA): anticapitalisme, lutte des classes
- Social-démocrate (PS, EELV): réformisme, justice sociale, écologie
- Centre (Renaissance, MoDem): pragmatisme, libéralisme modéré
- Droite (LR): conservatisme, libéralisme économique
- Extrême droite (RN, Reconquête): nationalisme, anti-immigration

## RÈGLES
1. Analyse CHAQUE speaker séparément
2. Si neutre → 20% chaque catégorie
3. Les pourcentages doivent totaliser 100%

## FORMAT JSON
{
    "speakers": {
        "0": {
            "role_probable": "<journaliste|invité|micro-trottoir|autre>",
            "description": "<brève description>",
            "extreme_gauche": <0-100>,
            "social_democrate": <0-100>,
            "centre": <0-100>,
            "droite": <0-100>,
            "extreme_droite": <0-100>,
            "ton": "<neutre|engagé|critique|sympathique|hostile>",
            "justification": "<1-2 phrases>"
        }
    },
    "analyse_globale": {
        "extreme_gauche": <0-100>,
        "social_democrate": <0-100>,
        "centre": <0-100>,
        "droite": <0-100>,
        "extreme_droite": <0-100>,
        "biais_editorial": "<neutre|léger|marqué|fort>",
        "justification": "<2-3 phrases>"
    }
}"""

print(f"\n📤 Envoi à GPT-4.1-mini...")

response = requests.post(
    'https://api.openai.com/v1/chat/completions',
    headers={
        'Authorization': f'Bearer {openai_key}',
        'Content-Type': 'application/json'
    },
    json={
        'model': 'gpt-4.1-mini',
        'messages': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': f"Cette transcription contient {num_speakers} speakers.\n\nAnalyse:\n\n{transcript[:100000]}"}
        ],
        'temperature': 0.3,
        'max_tokens': 2000
    },
    timeout=120
)
response.raise_for_status()

content = response.json()['choices'][0]['message']['content']

# Nettoyer le JSON
content = content.strip()
if content.startswith('```'):
    content = content.split('```')[1]
    if content.startswith('json'):
        content = content[4:]
content = content.strip()

analysis = json.loads(content)

# ========================================
# RÉSULTATS
# ========================================
print(f"\n{'='*60}")
print("RÉSULTATS DE L'ANALYSE")
print("="*60)

# Analyse globale
glob = analysis.get('analyse_globale', {})
print(f"\n📊 ORIENTATION POLITIQUE GLOBALE:")
print(f"   Extrême gauche:  {glob.get('extreme_gauche', 0):3}%")
print(f"   Social-démocrate:{glob.get('social_democrate', 0):3}%")
print(f"   Centre:          {glob.get('centre', 0):3}%")
print(f"   Droite:          {glob.get('droite', 0):3}%")
print(f"   Extrême droite:  {glob.get('extreme_droite', 0):3}%")
print(f"\n📰 Biais éditorial: {glob.get('biais_editorial', 'N/A')}")
print(f"💬 Justification: {glob.get('justification', 'N/A')}")

# Analyse par speaker
speakers = analysis.get('speakers', {})
print(f"\n👥 ANALYSE PAR SPEAKER ({len(speakers)} speakers):")
for speaker_id, info in speakers.items():
    print(f"\n   [Speaker {speaker_id}] - {info.get('role_probable', 'N/A')}")
    print(f"   Ton: {info.get('ton', 'N/A')}")
    print(f"   Orientation: EG={info.get('extreme_gauche',0)}% SD={info.get('social_democrate',0)}% C={info.get('centre',0)}% D={info.get('droite',0)}% ED={info.get('extreme_droite',0)}%")
    print(f"   → {info.get('justification', 'N/A')[:100]}")

# Sauvegarder le résultat complet
with open('test_result.json', 'w', encoding='utf-8') as f:
    json.dump({
        'mp3_url': TEST_MP3_URL,
        'transcript': transcript,
        'num_speakers': num_speakers,
        'analysis': analysis
    }, f, ensure_ascii=False, indent=2)

print(f"\n{'='*60}")
print("✅ Résultat complet sauvegardé dans: test_result.json")
print("="*60)